In [1]:
!pip install transformers==4.31.0
!pip install accelerate==0.21.0
!pip install bitsandbytes==0.40.2
!pip install peft==0.5.0
!pip install datasets
!pip install einops
!pip install torch==2.0.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0
    Uninstalling peft-0.4.0:
      Successfully uninstalled peft-0.4.0


In [3]:
!pip install fastapi uvicorn transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 470.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 3.1 MB/s eta 0:00:00a

In [5]:
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, LlamaForCausalLM
import torch
from pydantic import BaseModel

app = FastAPI()

# Load the fine-tuned model and tokenizer
model_name_or_path = "./myllama2model-ourdata-v2"  # Update this to your model path
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model = LlamaForCausalLM.from_pretrained(model_name_or_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")  # Move model to GPU if available

# Define request model for the API
class TextRequest(BaseModel):
    text: str
    max_new_tokens: int = 128

@app.post("/generate-text")
async def generate_text(request: TextRequest):
    """
    Endpoint to generate text based on input prompt.
    """
    # Tokenize input text
    inputs = tokenizer(request.text, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    
    # Generate text using the model
    outputs = model.generate(input_ids=inputs.input_ids, max_new_tokens=request.max_new_tokens)
    
    # Decode the generated tokens to text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Return the generated text
    return {"generated_text": generated_text}

if __name__ == "__main__":
    import uvicorn
    # Run the FastAPI application with uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


OSError: ./myllama2model-ourdata-v2 does not appear to have a file named config.json. Checkout 'https://huggingface.co/./myllama2model-ourdata-v2/main' for available files.